In [25]:
import pandas as pd
import numpy as np
import re
import pymorphy2 as pm
from sklearn.neighbors import LocalOutlierFactor
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, HashingVectorizer
from multiprocessing import Pool, Lock, Value
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score
from sklearn import svm

In [26]:
docs_titles = pd.read_csv('anomaly-detection-competition-ml1-ts-fall-2019/docs_titles.tsv', sep='\t', encoding='utf-8', lineterminator='\n')

In [28]:
docs_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27950 entries, 0 to 27949
Data columns (total 2 columns):
doc_id    27950 non-null int64
title     27919 non-null object
dtypes: int64(1), object(1)
memory usage: 436.8+ KB


In [29]:
ma = pm.MorphAnalyzer()

In [30]:
docs_titles['title'] = docs_titles['title'].astype(str)
docs_titles['title'] = [text.lower() for text in docs_titles['title']]
docs_titles['title'] = [re.sub('[^0-9a-zA-ZА-Яа-я]+', ' ', text) for text in docs_titles['title']]
def norm(text):
    res = ''
    for word in text.split():
        if word != '':
            t = ma.parse(word)[0].normal_form
            res = res + t
            res = res + ' '
    return res

with Pool(10) as pool:
    docs_titles['title'] = list(pool.map(norm, docs_titles['title']))

,doc_id,title
0,15731,ваза 21213 замена подшипник ступица нива
1,14829,ваза 2107 оптом в сочи сравнить цена купить по...
2,15764,купить ступица лада калина2 трансмиссия перехо...
3,17669,классика 21010 21074
4,14852,ступица нива замена подшипник свой рука
...,...,...
27945,16637,ответ mail ru полезно ли кушать творог по утро...
27946,16759,творог полезный свойство и лечение творог женс...
27947,15358,творог полезный и опасный свойство творог
27948,17287,ответ mail ru чем полезный творог


In [31]:
test_groups = pd.read_csv('anomaly-detection-competition-ml1-ts-fall-2019/test_groups.csv')

In [33]:
train_groups = pd.read_csv('anomaly-detection-competition-ml1-ts-fall-2019/train_groups.csv')

In [35]:
%%time
groups_train = []
target_groups = []
for group_id in range(1, 130):
    part = train_groups[train_groups.group_id == group_id]
    arr = []
    arr_t = []
    for doc_id in part['doc_id']:
        arr.append(docs_titles[docs_titles.doc_id == doc_id].iloc[0]['title'])
        arr_t.append(part[part.doc_id == doc_id].iloc[0]['target'])
    groups_train.append(arr)
    target_groups.append(arr_t)
groups_train[0]

CPU times: user 15.8 s, sys: 18.1 ms, total: 15.8 s
Wall time: 15.8 s


[['ваза 21213 замена подшипник ступица нива ',
  'ваза 2107 оптом в сочи сравнить цена купить потребительский товар на tiu ru ',
  'купить ступица лада калина2 трансмиссия переходный ступица цена замена тюнинг ',
  'классика 21010 21074 ',
  'ступица нива замена подшипник свой рука ',
  'ваза 2110 ',
  'обзор подшипник полуось ваза 2101 07 2121 2123 ',
  'купить подшипник и ступица fag страница 23 ',
  'horsepowers автомобильный интернет портал отзыв владелец ваза 2121 нива 2007 год ',
  'новость и сообщение из официальный группа вконтакте торговый компания 33 sport магазин тольятти ',
  'инструкция по замена подшипник передний ступица ивеко дейли через dorognoekam ru ',
  'ступица olx ua страница 80 ',
  'маааленький проблёмкий бортжурнал автокам 2160 1994 год на drive2 ',
  'разгрузить полуось для нива 24 шлиц 765 мм ',
  'прошивка для нива м7 9 7 скачать файлообменник emqraty6 foxkirov ru ',
  'страница 6 раздел каталог подвеска ',
  'продать нива 2121 ',
  'рекомендация по проведен

In [37]:
%%time
groups_test = []
for group_id in range(130, 310):
    part = test_groups[test_groups.group_id == group_id]
    arr = []
    for doc_id in part['doc_id']:
        if len(docs_titles[docs_titles.doc_id == doc_id]) == 0:
            arr.append('')
        else:
            arr.append(docs_titles[docs_titles.doc_id == doc_id].iloc[0]['title'])
    groups_test.append(arr)
groups_test[0]

CPU times: user 21.9 s, sys: 31.6 ms, total: 22 s
Wall time: 22 s


['как прописать админк в кс 1 6 себя или друг youtube ',
 'скачать sgl rp доработка слив мода mysql rp role play готовый сервер для samp 0 3 7 0 3z 0 3x 0 3e v sampe ru вс для samp и gta ',
 'как прописать админк в кс 1 6 counter strike каталог стать игровой сообщество dream x ru counter strike портал ',
 'как прописать простой админк в кс 1 6 ',
 'подбор админовый для сервер по код 4 архив форум ozone ',
 'каталог стать the best original portal in ukraine the best original portal in ukrain ',
 'eugene kirian блог ',
 'файл htaccess основной параметр увеличивать безопасность блог ',
 'как дать себя админк в cs 1 6 на хостинг my arena ответ на игра вокруг свет и к другой игра ',
 'joomla 2 5 убирать хлебный крошка с выбрать страница ',
 'sa mp беседка архив страница 3 форум ozone ',
 'чужой wifi форум на исходник ру ',
 'спасибо за комментарий с плагин и без он ',
 'cs amxmodmenu как прописать админк ',
 'пароль на admin hack 3 7 в c s 1 6 ',
 'как пользоваться сервер в кс видео ',
 'ка

In [39]:
cv = CountVectorizer(min_df=2, max_df=0.8)
tf = TfidfTransformer()
matrix_train = []
for group in groups_train:
    m = cv.fit_transform(group).toarray()
    matrix_train.append(tf.fit_transform(m).toarray())

In [41]:
matrix_test = []
for group in groups_test:
    m = cv.fit_transform(group).toarray()
    matrix_test.append(tf.fit_transform(m).toarray())

In [43]:
magic_forest = IsolationForest(n_estimators=300, behaviour='new', contamination='auto')
all_targ = np.array(train_groups['target'])
n, c = np.unique(all_targ, return_counts=True)
n, c

(array([0, 1]), array([8329, 3361]))

In [44]:
scores =[]
preds = []
for k in ['rbf', 'poly', 'linear', 'sigmoid']:
    for degree in range(1, 10):
        for nu in [0.3, 0.4, 0.5, 0.6, 0.7]:
            y_pred = []
            magic_forest = svm.OneClassSVM(kernel=k, degree=degree, gamma='scale', nu=nu)
            for m in matrix_train:
                y = magic_forest.fit_predict(m)
                for i in range(y.size):
                    if y[i] == -1:
                        y[i] = 0
                y =list(y)
                y_pred += y
            score = f1_score(y_pred, all_targ)
            scores.append(score)
            preds.append(y_pred)
            print('DONE FOR KERNEL = {} DEGREE = {} NU = {}, SCORE = {}'.format(k, degree, nu, score))

DONE FOR KERNEL = rbf DEGREE = 1 NU = 0.3, SCORE = 0.4117306579201234
DONE FOR KERNEL = rbf DEGREE = 1 NU = 0.4, SCORE = 0.4025044722719141
DONE FOR KERNEL = rbf DEGREE = 1 NU = 0.5, SCORE = 0.3947564744751146
DONE FOR KERNEL = rbf DEGREE = 1 NU = 0.6, SCORE = 0.3715834118755891
DONE FOR KERNEL = rbf DEGREE = 1 NU = 0.7, SCORE = 0.34838709677419355
DONE FOR KERNEL = rbf DEGREE = 2 NU = 0.3, SCORE = 0.4117306579201234
DONE FOR KERNEL = rbf DEGREE = 2 NU = 0.4, SCORE = 0.4025044722719141
DONE FOR KERNEL = rbf DEGREE = 2 NU = 0.5, SCORE = 0.3947564744751146
DONE FOR KERNEL = rbf DEGREE = 2 NU = 0.6, SCORE = 0.3715834118755891
DONE FOR KERNEL = rbf DEGREE = 2 NU = 0.7, SCORE = 0.34838709677419355
DONE FOR KERNEL = rbf DEGREE = 3 NU = 0.3, SCORE = 0.4117306579201234
DONE FOR KERNEL = rbf DEGREE = 3 NU = 0.4, SCORE = 0.4025044722719141
DONE FOR KERNEL = rbf DEGREE = 3 NU = 0.5, SCORE = 0.3947564744751146
DONE FOR KERNEL = rbf DEGREE = 3 NU = 0.6, SCORE = 0.3715834118755891
DONE FOR KERNEL = 

DONE FOR KERNEL = linear DEGREE = 6 NU = 0.3, SCORE = 0.4644619038918327
DONE FOR KERNEL = linear DEGREE = 6 NU = 0.4, SCORE = 0.44494163424124517
DONE FOR KERNEL = linear DEGREE = 6 NU = 0.5, SCORE = 0.4393151278716949
DONE FOR KERNEL = linear DEGREE = 6 NU = 0.6, SCORE = 0.42106570612346295
DONE FOR KERNEL = linear DEGREE = 6 NU = 0.7, SCORE = 0.3927648578811369
DONE FOR KERNEL = linear DEGREE = 7 NU = 0.3, SCORE = 0.4644619038918327
DONE FOR KERNEL = linear DEGREE = 7 NU = 0.4, SCORE = 0.44494163424124517
DONE FOR KERNEL = linear DEGREE = 7 NU = 0.5, SCORE = 0.4393151278716949
DONE FOR KERNEL = linear DEGREE = 7 NU = 0.6, SCORE = 0.42106570612346295
DONE FOR KERNEL = linear DEGREE = 7 NU = 0.7, SCORE = 0.3927648578811369
DONE FOR KERNEL = linear DEGREE = 8 NU = 0.3, SCORE = 0.4644619038918327
DONE FOR KERNEL = linear DEGREE = 8 NU = 0.4, SCORE = 0.44494163424124517
DONE FOR KERNEL = linear DEGREE = 8 NU = 0.5, SCORE = 0.4393151278716949
DONE FOR KERNEL = linear DEGREE = 8 NU = 0.6, 

In [45]:
y_pred = preds[np.argmax(scores)]
print(np.max(scores))
u,c = np.unique(y_pred, return_counts=True)

0.4644619038918327


In [46]:
c, u

(array([4105, 7585]), array([0, 1]))

Isolation forest я решил убрать, так как с ним скор 0.3